In [1]:
%matplotlib inline

In [2]:
import serial, threading, sys, time
sys.path.append('/home/pi/Desktop/ADR_NX/Utils')
sys.path.append('/home/pi/Desktop/ADR_NX/Mainframe')
from adr_PID import ADR_PID
import adr_utils, SIM900_controller, SIM922
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [3]:
mfc = SIM900_controller.SIM900()
mfc.debug = False
mfc.init_and_open_connection()
s922 = SIM922.SIM922(mfc,'s922',debug=True);
mfc.set_RPER([s922.MODNUM])

In [4]:
mfc.start_input_loop()
mfc.start_output_loop()

Input loop started
Output loop started


In [5]:
from IPython.display import clear_output
temps, times = ([] for i in range(2))
s = timer()

mfc.debug = False
s922.dbg = False
s922.start_data_stream(0)
i = 0
try:
    while timer() - s < (24*60*60):
        s922.newdataevent.clear()
        s922.newdataevent.wait(5)
        s922.newdataevent.clear()
        temp = s922.lastvalues
        tm = s922.lastdatatimes
                   
        sys.stdout.flush()
        sys.stderr.flush()
        try:
            print s922.excq.get_nowait()
            print "OOPS!"
        except:
            #print "No exceptions!"
            pass                
        #print("|{:5d}|{:s}|{:8f}".format(i,temp,tm[0]))
        temps.append(list(temp));
        times.append(list(tm));
        i+=1
        time.sleep(5)
except Exception as e:
    print e
finally:
    s922.stop_data_stream()
    s922.flush_queue()

HI


SIM900_controller.py:194: UserWarning: Bad message payload length - msg:|'MSG 6,#2545.232800E+01, 5.653000E+00, 5.723000E+00, 1.376550E+02'| len:|58| (want |52|)
  warnings.warn("Bad message payload length - msg:|{}| len:|{}| (want |{}|)".format(repr(msg),len(msg2),numchar))
SIM900_controller.py:194: UserWarning: Bad message payload length - msg:|'MSG 6,#2545.232000E+01, 5.654000E+00, 5.701000E+00, 1.378170E+02'| len:|58| (want |52|)
  warnings.warn("Bad message payload length - msg:|{}| len:|{}| (want |{}|)".format(repr(msg),len(msg2),numchar))
SIM900_controller.py:194: UserWarning: Bad message payload length - msg:|'MSG 6,#2545.228800E+01, 5.653000E+00, 5.664000E+00, 1.377240E+02'| len:|58| (want |52|)
  warnings.warn("Bad message payload length - msg:|{}| len:|{}| (want |{}|)".format(repr(msg),len(msg2),numchar))
SIM900_controller.py:194: UserWarning: Bad message payload length - msg:|'MSG 6,#2545.216900E+01, 5.652000E+00, 5.679000E+00, 1.376530E+02'| len:|58| (want |52|)
  warnings

KeyboardInterrupt: 

In [ ]:
data1 = [[x[i] for x in temps] for i in range(0,4)]

In [ ]:
fig = plt.figure()
plt.plot(times,data1[0],times,data1[1],times,data1[2],times,data1[3],'-')
plt.xlabel("Time (s)")
plt.ylabel("Temperature (K)")
plt.show()

In [ ]:
with open("warmup_20170710.txt", "w") as f:
    for i in range(0,len(times)):
        f.write("{} {} {}\n".format(times[i],temps[i],dcs[i]))

In [ ]:
mfc.stop_input_loop()
mfc.stop_output_loop()

In [6]:
len('5.232800E+01, 5.653000E+00, 5.723000E+00, 1.376550E+02')

54

In [ ]:
range(0,4)

In [ ]:
mfc.